In [4]:
!pip install langchain langchain-openai langchain-community langchain-core openai pypdf azure-keyvault-secrets azure-identity faiss-cpu dotenv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
           ---------------------------------------- 0.0/145.4 kB ? eta -:--:--
           ---------- -------------------------- 41.0/145.4 kB 991.0 kB/s eta 0:00:01
           -------------------------------------- 145.4/145.4 kB 2.2 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— Preparing metadata (pyproject.toml) did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [6 lines of output]
            usage: setup.py [global

In [23]:
import json
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import LLMChain
from langchain_openai import AzureChatOpenAI

In [24]:
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
from openai import AzureOpenAI

deployment_name = "uptale-gpt-35-turbo"

llm = AzureChatOpenAI(
    azure_deployment=deployment_name
)

In [26]:
def process_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    return texts

In [27]:
def generate_title(texts):
    combined_text = " ".join(text.page_content for text in texts)
    prompt = PromptTemplate(
        input_variables=["document"],
        template="Create a catchy title for a 360-degree educational experience based on it."
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(document=combined_text)
    return response.strip()

In [28]:
def generate_scenes(texts, num_scenes=5):
    combined_text = " ".join(text.page_content for text in texts)
    scenes = []
    for i in range(num_scenes):
        prompt = PromptTemplate(
            input_variables=["scene_number", "document"],
            template="""
            Create scene {scene_number} for a 360-degree educational experience based on the document content.
            Include:
            1. A brief description of the scene
            2. Educational content relevant to the document
            3. A transition to the next scene (use one of the following):
               - Tag Door => Target Scene
               - Tag QCM => Go to Scene (with up to 4 multiple-choice answers)
               - Tag Microphone Rules => Allow to trigger an action to go to another scene

            Format the output as a JSON object.
            """
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        scene = chain.run(scene_number=i+1, document=combined_text)
        try:
            scenes.append(json.loads(scene))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for scene {i+1}. Skipping this scene.")
    return scenes

In [29]:
def generate_360_experience(pdf_path):
    texts = process_pdf(pdf_path)
    title = generate_title(texts)
    scenes = generate_scenes(texts)

    experience = {
        "title": title,
        "scenes": scenes
    }

    return json.dumps(experience, indent=2)

In [14]:
pdf_path = "./content/example1.pdf"
experience_json = generate_360_experience(pdf_path)
print(experience_json)

with open("./content/360_experience.json", "w") as f:
    f.write(experience_json)

{
  "title": "\"Revolutionizing Education: A 360-Degree Journey towards Limitless Learning\"",
  "scenes": [
    {
      "scene1": {
        "description": "Scene 1 takes place in a virtual classroom. The room is equipped with interactive whiteboards, desks with tablets, and a teacher's desk at the front.",
        "educationalContent": "The document discusses the benefits of using technology in education. It highlights how interactive whiteboards and tablets can enhance student engagement and learning outcomes.",
        "transition": {
          "tag": "Tag Door",
          "targetScene": "scene2"
        }
      }
    },
    {
      "scene2": {
        "description": "Scene 2: The Rainforest",
        "educationalContent": "In this scene, the user is transported to a lush rainforest. They are surrounded by tall trees, vibrant flowers, and the sound of chirping birds. The immersive experience allows the user to explore the different layers of the rainforest ecosystem and learn about 

 # CSM Agent

In [37]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent

In [38]:
def generate_title_tool(texts, csm_instructions):
    combined_text = " ".join(text.page_content for text in texts)
    prompt = PromptTemplate(
        input_variables=["document", "instructions"],
        template="Create a catchy title for a 360-degree educational experience based on the following document, while adhering to these instructions: {instructions}\n\nDocument: {document}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(document=combined_text, instructions=csm_instructions)
    return response.strip()

def generate_scenes_tool(texts, csm_instructions, num_scenes=5):
    combined_text = " ".join(text.page_content for text in texts)
    scenes = []
    for i in range(num_scenes):
        prompt = PromptTemplate(
            input_variables=["scene_number", "document", "instructions"],
            template="""
            Create scene {scene_number} for a 360-degree educational experience based on the document content, while adhering to these instructions: {instructions}

            Include:
            1. A brief description of the scene
            2. Educational content relevant to the document
            3. A transition to the next scene (use one of the following):
               - Tag Door => Target Scene
               - Tag QCM => Go to Scene (with up to 4 multiple-choice answers)
               - Tag Microphone Rules => Allow to trigger an action to go to another scene

            Format the output as a JSON object.
            """
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        scene = chain.run(scene_number=i+1, document=combined_text, instructions=csm_instructions)
        try:
            scenes.append(json.loads(scene))
        except json.JSONDecodeError:
            print(f"Error parsing JSON for scene {i+1}. Skipping this scene.")
    return scenes

def evaluate_experience_tool(experience, csm_instructions):
    prompt = PromptTemplate(
        input_variables=["experience", "instructions"],
        template="""
        Evaluate the following 360-degree educational experience based on these instructions: {instructions}

        Experience:
        {experience}

        Provide a score from 1 to 10 and a brief explanation of your evaluation.
        Return your response as a JSON object with 'score' and 'explanation' keys.
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    evaluation = chain.run(experience=experience, instructions=csm_instructions)
    return json.loads(evaluation)


In [43]:
from langchain.prompts import StringPromptTemplate
from typing import List
from pydantic import BaseModel, Field

class CSMAgentPromptTemplate(StringPromptTemplate, BaseModel):
    template: str = Field(default="""You are an AI assistant tasked with creating and evaluating a 360-degree educational experience based on provided content and instructions.

Available tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}""")

    input_variables: List[str] = Field(default=["input", "agent_scratchpad", "tools", "tool_names"])

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps", [])
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in kwargs["tools"]])
        kwargs["tool_names"] = ", ".join([tool.name for tool in kwargs["tools"]])
        return self.template.format(**kwargs)

In [47]:

def run_csm_agent(pdf_path, csm_instructions_path):
    loader = PyPDFLoader(pdf_path)
    texts = loader.load_and_split()
    
    with open(csm_instructions_path, 'r') as file:
        csm_instructions = file.read()

    tools = [
        Tool(
            name="GenerateTitle",
            func=lambda: generate_title_tool(texts, csm_instructions),
            description="Generates a title for the 360-degree experience"
        ),
        Tool(
            name="GenerateScenes",
            func=lambda: generate_scenes_tool(texts, csm_instructions),
            description="Generates scenes for the 360-degree experience"
        ),
        Tool(
            name="EvaluateExperience",
            func=lambda experience: evaluate_experience_tool(experience, csm_instructions),
            description="Evaluates the generated 360-degree experience"
        )
    ]


    prompt = CSMAgentPromptTemplate()

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    agent = LLMSingleActionAgent(llm_chain=llm_chain, tools=tools)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    experience = None
    evaluation = {"score": 0}

    while evaluation["score"] < 7:
        experience = {}
        
        experience["title"] = agent_executor.run("Generate a title for the 360-degree educational experience.")
        
        experience["scenes"] = agent_executor.run("Generate 5 scenes for the 360-degree educational experience.")
        
        evaluation_result = agent_executor.run(f"Evaluate the following 360-degree educational experience:\n{json.dumps(experience)}")
        evaluation = json.loads(evaluation_result)
        
        print(f"Current evaluation score: {evaluation['score']}")
        
        if evaluation["score"] < 7:
            print("Score below 7. Regenerating the experience...")
        else:
            print("Score 7 or above. Returning the final experience.")

    return experience

In [48]:
result = run_csm_agent("./content/example1.pdf", "./content/csm_instructions.txt")
print(result)

ValidationError: 1 validation error for CSMAgentPromptTemplate
input_variables
  field required (type=value_error.missing)